In [4]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [5]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [6]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [7]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

In [8]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [10]:
runnable.invoke({"input": "hi!"})

{'input': 'hi!', 'chat_history': []}

In [11]:
chain = runnable | prompt | model

In [12]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 테디입니다."})
print(response.content)  # 생성된 응답을 출력합니다.

만나서 반가워요, 테디님! 무엇을 도와드릴까요?


In [13]:
memory.load_memory_variables({})

{'chat_history': []}

In [14]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 테디입니다."}, {"ai": response.content}
)

# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 테디입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='만나서 반가워요, 테디님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

In [15]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)

네, 당신의 이름은 테디입니다! 어떤 도움이 필요하신가요?


In [16]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 초기화합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [17]:
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력합니다.
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer

In [18]:
# ChatOpenAI 모델을 초기화합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 요약 메모리로 교체할 경우
# memory = ConversationSummaryMemory(
#     llm=llm, return_messages=True, memory_key="chat_history"
# )

conversation_chain = MyConversationChain(llm, prompt, memory)

In [19]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.")

'안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [20]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'테디님이라고 하셨어요! 어떻게 도와드릴까요?'

In [21]:
conversation_chain.invoke("앞으로는 영어로만 답변해주세요 알겠어요?")

'Understood! I will respond in English from now on. How can I assist you today?'

In [22]:
conversation_chain.invoke("제 이름을 다시 한 번 말해주세요")

'Your name is Teddy. How can I assist you further?'

In [23]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐라고요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='테디님이라고 하셨어요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='앞으로는 영어로만 답변해주세요 알겠어요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Understood! I will respond in English from now on. How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 다시 한 번 말해주세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Teddy. How can I assist you further?', additional_kwargs={}, response_metadata={})]